In [1]:
import pandas as pd
import numpy as np

In [2]:
#Import email snippets
emails = pd.read_csv('email_outcome.csv')

In [3]:
emails.head()

,Unnamed: 0,email,outcome
0,0,"Hi All, We still have openings for lunch with ...",2
1,1,"Hello Strad Slater , Thank you for your intere...",1
2,2,Your inside source for insurance coverage Your...,2
3,3,You don&#39;t want to miss this. ADICTIVA PART...,2
4,4,Hello Polymaths ~ I wanted to share with you a...,2


In [4]:
emails.drop(columns='Unnamed: 0', inplace=True)

In [5]:
emails.head()

,email,outcome
0,"Hi All, We still have openings for lunch with ...",2
1,"Hello Strad Slater , Thank you for your intere...",1
2,Your inside source for insurance coverage Your...,2
3,You don&#39;t want to miss this. ADICTIVA PART...,2
4,Hello Polymaths ~ I wanted to share with you a...,2


In [6]:
emails['email']

0      Hi All, We still have openings for lunch with ...
1      Hello Strad Slater , Thank you for your intere...
2      Your inside source for insurance coverage Your...
3      You don&#39;t want to miss this. ADICTIVA PART...
4      Hello Polymaths ~ I wanted to share with you a...
                             ...                        
995    RSVP NOW! SUNDAY, SEPTEMBER 3RD Join us tonigh...
996    Applicant Satisfaction Survey Hi Strad Slater ...
997    Hello, Here is my proposed schedule for next s...
998    Hello Strad, Following up to confirm that your...
999    Department of Physics and Astronomy Emergent C...
Name: email, Length: 1000, dtype: object

In [7]:
import re 
import re
import nltk
import spacy
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize

nltk.download('stopwords')
nltk.download('punkt')
nlp = spacy.load('en_core_web_sm')

#Create a function to remove stop words and clean up the test
def clean_text(text, use_stemming=True):
    # Remove punctuation and special characters
    text = re.sub(r'[^A-Za-z\s]', '', text)

    # Convert text to lowercase
    text = text.lower()

    # Tokenize the text
    words = word_tokenize(text)

    # Remove stop words
    stop_words = set(stopwords.words('english'))
    words = [word for word in words if word not in stop_words]

    if use_stemming:
        # Apply stemming
        stemmer = PorterStemmer()
        words = [stemmer.stem(word) for word in words]
    else:
        # Apply lemmatization
        words = [nlp(word)[0].lemma_ for word in words]

    return  words

#Converts outcome to a 1 or 0 value
def clean_outcome(num):
    if num == 2:
        return 0
    else:
        return num

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\17028\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\17028\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
C:\Users\17028\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
emails['email_clean'] = emails['email'].apply(lambda x: clean_text(x))

In [9]:
emails

,email,outcome,email_clean
0,"Hi All, We still have openings for lunch with ...",2,"[hi, still, open, lunch, colloquium, speaker, ..."
1,"Hello Strad Slater , Thank you for your intere...",1,"[hello, strad, slater, thank, interest, bioban..."
2,Your inside source for insurance coverage Your...,2,"[insid, sourc, insur, coverag, insid, sourc, i..."
3,You don&#39;t want to miss this. ADICTIVA PART...,2,"[dont, want, miss, adictiva, parti, insid, kul..."
4,Hello Polymaths ~ I wanted to share with you a...,2,"[hello, polymath, want, share, upcom, polymath..."
...,...,...,...
995,"RSVP NOW! SUNDAY, SEPTEMBER 3RD Join us tonigh...",2,"[rsvp, sunday, septemb, rd, join, us, tonight,..."
996,Applicant Satisfaction Survey Hi Strad Slater ...,1,"[applic, satisfact, survey, hi, strad, slater,..."
997,"Hello, Here is my proposed schedule for next s...",2,"[hello, propos, schedul, next, semest, math, m..."
998,"Hello Strad, Following up to confirm that your...",2,"[hello, strad, follow, confirm, onboard, compl..."


In [10]:
emails['outcome'] = emails['outcome'].apply(lambda x: clean_outcome(x))

In [11]:
emails.head()

,email,outcome,email_clean
0,"Hi All, We still have openings for lunch with ...",0,"[hi, still, open, lunch, colloquium, speaker, ..."
1,"Hello Strad Slater , Thank you for your intere...",1,"[hello, strad, slater, thank, interest, bioban..."
2,Your inside source for insurance coverage Your...,0,"[insid, sourc, insur, coverag, insid, sourc, i..."
3,You don&#39;t want to miss this. ADICTIVA PART...,0,"[dont, want, miss, adictiva, parti, insid, kul..."
4,Hello Polymaths ~ I wanted to share with you a...,0,"[hello, polymath, want, share, upcom, polymath..."


In [12]:
#Create a dictionary of all the words used across the snipets
dictionary = sum(emails['email_clean'], [])

In [13]:
from collections import Counter

#Count how many unique words are in the dictionary
counter = Counter(dictionary)
len(counter)

3044

In [14]:
#Create train and testing datasets and encode each email snipet into vectors showing if they contain each word in the dictionary
encoding = {}
train_emails = emails.loc[0:700]
test_emails = emails.loc[701:999]

for key in counter.keys():
    encoding[key] = []

for key in counter.keys():
    for i in range(0, len(train_emails)):
        if key in train_emails['email_clean'].values[i]:
            encoding[key].append(1)
        else:
            encoding[key].append(0)

In [15]:
features = pd.DataFrame(encoding)

In [16]:
features['[[Result]]'] = emails['outcome']

In [17]:
features

,hi,still,open,lunch,colloquium,speaker,rehan,kapadia,usc,would,...,onboard,aziz,beat,cilia,swim,fish,eva,kanso,aerospac,[[Result]]
0,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
696,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
697,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
698,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
699,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [18]:
#Calculate needed probabilities for Naive Bayes
amount_job = len(features[features['[[Result]]'] == 1])
amount_no_job = len(features[features['[[Result]]'] == 0])

prob_word_job = {}
prob_word_no_job = {}

for key in counter.keys():
    prob_word_job[key] = (len(features[(features[key] == 1) & (features['[[Result]]'] == 1)])+ 1)/(amount_job + 2)
    prob_word_no_job[key] = (len(features[(features[key] == 1) & (features['[[Result]]'] == 0)]) + 1)/(amount_no_job + 2)


prob_job = amount_job/len(train_emails)
prob_no_job = 1 - prob_job

In [27]:
#Create a function that predicts the email snippets category based on the Naive Bayes model
def predict(sentence, threshold):   
    word_product = 1
    no_word_product = 1

    for word in sentence:
        word_product *= prob_word_job[word]
        no_word_product *= prob_word_no_job[word]
    word_in_class = (word_product*prob_job)/((word_product*prob_job)+(no_word_product*prob_no_job))
    word_no_in_class = (no_word_product*prob_no_job)/((word_product*prob_job)+(no_word_product*prob_no_job))

    if (word_in_class >= threshold) & (word_in_class >= word_no_in_class):
        return 1
    else:
        return 0

In [28]:
test_emails['predict'] = test_emails['email_clean'].apply(lambda x: predict(x, 0.9999))

In [29]:
#Calculate total accuracy and accuracy for each class
accuracy = test_emails['outcome'] - test_emails['predict']
accuracy = accuracy.to_list()
accuracy = accuracy.count(0)/len(accuracy)

job_accuracy = test_emails[test_emails['outcome'] == 1]['outcome'] - test_emails[test_emails['outcome'] == 1]['predict']
job_accuracy = job_accuracy.to_list()
job_accuracy = job_accuracy.count(0)/len(job_accuracy)

no_job_accuracy = test_emails[test_emails['outcome'] == 0]['outcome'] - test_emails[test_emails['outcome'] == 0]['predict']
no_job_accuracy = no_job_accuracy.to_list()
no_job_accuracy = no_job_accuracy.count(0)/len(no_job_accuracy)

print('Total Accuracy: ', accuracy)
print('Job Accuracy: ', job_accuracy)
print('Not Job Related Accuracy: ', no_job_accuracy)

Total Accuracy:  0.8662207357859532
Job Accuracy:  0.9782608695652174
Not Job Related Accuracy:  0.8458498023715415


In [24]:
#Job category predictions
pd.set_option('display.max_colwidth', None)
test_emails[(test_emails['predict'] == 1)]['email'].head()

701    Hi Strad, Ready to dive into the world of numbers, data, and innovation? 🎲 Arch Reinsurance Actuarial Internship in Morristown, NJ is your ticket to an immersive actuarial experience! 🌐 🏢 About Arch Re
706           Dear Strad, Thank you for submitting your application for the position of Co-op, Small Molecule Analytical Development and your interest in Biogen. You have successfully completed the submission
708      Tripp Carrington III shared a document Tripp Carrington III (kwcarrin@usc.edu) has invited you to edit the following document: T Men Speech Topics Open Google LLC, 1600 Amphitheatre Parkway, Mountain
714       Grainger Businesses Logo Thank you for joining the Grainger Businesses talent community! Joining our talent community will allow us to notify you directly when roles aligned to your interest present
718        This notification was automatically generated by Workday, Autodesk&#39;s HR system. Please do not reply directly to this email. Hello Strad , We&#39;re t

In [25]:
#Non-job category predictions
test_emails[(test_emails['predict'] == 0)]['email'].head()

702          USC School of Cinematic Arts SCA Events War, Justice, and Democracy in the Films of Steven Spielberg Presents: BRIDGE OF SPIES (2015) Make Reservations » November 13, 2023, 7:00 PM Norris Cinema
703                                                                                                                                               Here is my Rough Draft for the Ethics Paper. Thank you, Strad
704    There is significant activity at the center of the UPC campus due to a demonstration. There may be difficulty accessing that area. USC Department of Public Safety University Park Campus (UPC) 213-740-
705       Let&#39;s get togetherJoin us for ADICTIVA PARTY PRESENTS &quot;NOCHE DE FUMAR&quot; |EVERYONE $5 BEFORE 10:30 w/ RSVP, April 20, 2024 at 10:00 PM.Register soon because space is limited.We hope you
707     You don&#39;t want to miss this. STOP LIGHT PARTY INSIDE CATCHONE LA 18+ FRIDAY DECEMBER 1ST &quot;STOP LIGHT PARTY HOSTED BY UCLA&quot; INSIDE CATCHONE LA (UND